In [145]:
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import chart_studio.plotly as py
import plotly.express as px
from collections import Counter
import pandas as pd
import collections
from collections import Counter
import matplotlib.pyplot as plt
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import lxml
from urllib.request import urlopen
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
import os
import plotly.graph_objects as go

So for this file we are going to prepare the data to be used in a model in further notebooks to help predict what type of team 'profile' is needed to win the Premier League. 

In [146]:
Link = "https://www.marca.com/en/football/premier-league/winners.html"

In [147]:
response = requests.get(Link)

response.raise_for_status()  # Check for request errors

# Parse the HTML content
soup = BeautifulSoup(response.text, 'html.parser')

In [148]:
table = soup.find('table')

In [149]:
table

<table cellpadding="0" cellspacing="0" class="ue-table-record" summary="Premier League Winners">
<caption class="ue-table-record__caption">
Premier League Winners
</caption>
<thead class="ue-table-record__thead">
<tr class="ue-table-record__tr">
<th class="ue-table-record__th" scope="col">Year</th>
<th class="ue-table-record__th is-marked" scope="col">Winner</th>
<th class="ue-table-record__th" scope="col">Runners-up</th>
</tr>
</thead>
<tbody class="ue-table-record__tbody">
<tr class="ue-table-record__tr">
<th class="ue-table-record__th" scope="row"><span class="ue-table-record__year">2024-25</span></th>
<td class="ue-table-record__td is-main" scope="row"><span class="ue-table-record__name is-marked">Liverpool</span></td>
<td class="ue-table-record__td"><span class="ue-table-record__name">Arsenal</span></td>
</tr>
<tr class="ue-table-record__tr">
<th class="ue-table-record__th" scope="row"><span class="ue-table-record__year">2023-24</span></th>
<td class="ue-table-record__td is-main" 

In [150]:
rows  = table.find_all('tr')  # Find all rows in the table
rows

[<tr class="ue-table-record__tr">
 <th class="ue-table-record__th" scope="col">Year</th>
 <th class="ue-table-record__th is-marked" scope="col">Winner</th>
 <th class="ue-table-record__th" scope="col">Runners-up</th>
 </tr>,
 <tr class="ue-table-record__tr">
 <th class="ue-table-record__th" scope="row"><span class="ue-table-record__year">2024-25</span></th>
 <td class="ue-table-record__td is-main" scope="row"><span class="ue-table-record__name is-marked">Liverpool</span></td>
 <td class="ue-table-record__td"><span class="ue-table-record__name">Arsenal</span></td>
 </tr>,
 <tr class="ue-table-record__tr">
 <th class="ue-table-record__th" scope="row"><span class="ue-table-record__year">2023-24</span></th>
 <td class="ue-table-record__td is-main" scope="row"><span class="ue-table-record__name is-marked">Manchester City</span></td>
 <td class="ue-table-record__td"><span class="ue-table-record__name">Arsenal</span></td>
 </tr>,
 <tr class="ue-table-record__tr">
 <th class="ue-table-record__

In [151]:

Years = []
Winners = []
for row in rows:
    cells = row.find_all(['td', 'th'])
    cell_data = [cell.get_text(strip=True) for cell in cells]
    Years.append(cell_data[0])
    Winners.append(cell_data[1])
    #cell_data_array.append(cell_data)
    print(cell_data)

['Year', 'Winner', 'Runners-up']
['2024-25', 'Liverpool', 'Arsenal']
['2023-24', 'Manchester City', 'Arsenal']
['2022-23', 'Manchester City', 'Arsenal']
['2021-22', 'Manchester City', 'Liverpool']
['2020-21', 'Manchester City', 'Manchester United']
['2019-20', 'Liverpool', 'Manchester City']
['2018-19', 'Manchester City', 'Liverpool']
['2017-18', 'Manchester City', 'Manchester United']
['2016-17', 'Chelsea', 'Tottenham Hotspur']
['2015-16', 'Leicester City', 'Arsenal']
['2014-15', 'Chelsea', 'Manchester City']
['2013-14', 'Manchester City', 'Liverpool']
['2012-13', 'Manchester United', 'Manchester City']
['2011-12', 'Manchester City', 'Manchester United']
['2010-11', 'Manchester United', 'Chelsea']
['2009-10', 'Chelsea', 'Manchester United']
['2008-09', 'Manchester United', 'Liverpool']
['2007-08', 'Manchester United', 'Chelsea']
['2006-07', 'Manchester United', 'Chelsea']
['2005-06', 'Chelsea', 'Manchester United']
['2004-05', 'Chelsea', 'Arsenal']
['2003-04', 'Arsenal', 'Chelsea']
['

In [152]:
Years = Years[1:]
Winners = Winners[1:]

In [153]:
pastwinners = pd.DataFrame({"Year": Years, "Winner": Winners})
pastwinners

,Year,Winner
0,2024-25,Liverpool
1,2023-24,Manchester City
2,2022-23,Manchester City
3,2021-22,Manchester City
4,2020-21,Manchester City
...,...,...
132,1892-93,Sunderland
133,1891-92,Sunderland
134,1890-91,Everton
135,1889-90,Preston North End


In [154]:
len(pastwinners)

137

In [155]:
pastwinners.iloc[3:25]

,Year,Winner
3,2021-22,Manchester City
4,2020-21,Manchester City
5,2019-20,Liverpool
6,2018-19,Manchester City
7,2017-18,Manchester City
8,2016-17,Chelsea
9,2015-16,Leicester City
10,2014-15,Chelsea
11,2013-14,Manchester City
12,2012-13,Manchester United


In [156]:
pastwinners['Winner'] = pastwinners['Winner'].replace('Manchester City','Man City')
pastwinners['Winner'] = pastwinners['Winner'].replace('Manchester United','Man United')

In [157]:
count = Counter(pastwinners['Winner'])
count

Counter({'Liverpool': 20,
         'Man United': 20,
         'Arsenal': 13,
         'Man City': 10,
         'Everton': 9,
         'Aston Villa': 7,
         'Chelsea': 6,
         'It was not played because of World War II': 6,
         'Sunderland': 6,
         'Newcastle United': 4,
         'It was not played because of World War I': 4,
         'Blackburn Rovers': 3,
         'Leeds United': 3,
         'Wolverhampton Wanderers': 3,
         'Huddersfield Town': 3,
         'Derby County': 2,
         'Tottenham Hotspur': 2,
         'Burnley': 2,
         'Portsmouth': 2,
         'Sheffield Wednesday': 2,
         'The Wednesday': 2,
         'Preston North End': 2,
         'Leicester City': 1,
         'Nottingham Forest': 1,
         'Ipswich Town': 1,
         'Discontinued due to World War II': 1,
         'West Bromwich Albion': 1,
         'Sheffield United': 1})

In [158]:
Season1 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2000-01.csv") 
ListofTeams = Season1['HomeTeam'].unique().tolist()
ListofTeams

['Charlton',
 'Chelsea',
 'Coventry',
 'Derby',
 'Leeds',
 'Leicester',
 'Liverpool',
 'Sunderland',
 'Tottenham',
 'Man United',
 'Arsenal',
 'Bradford',
 'Ipswich',
 'Middlesbrough',
 'Everton',
 'Man City',
 'Newcastle',
 'Southampton',
 'West Ham',
 'Aston Villa']

In [159]:
for i in range(len(ListofTeams)):
    weightofwinning = count[ListofTeams[i]] / len(pastwinners) * 100
    print(f"Ratio of winning the Premier League for {ListofTeams[i]}: {weightofwinning}")


Ratio of winning the Premier League for Charlton: 0.0
Ratio of winning the Premier League for Chelsea: 4.37956204379562
Ratio of winning the Premier League for Coventry: 0.0
Ratio of winning the Premier League for Derby: 0.0
Ratio of winning the Premier League for Leeds: 0.0
Ratio of winning the Premier League for Leicester: 0.0
Ratio of winning the Premier League for Liverpool: 14.5985401459854
Ratio of winning the Premier League for Sunderland: 4.37956204379562
Ratio of winning the Premier League for Tottenham: 0.0
Ratio of winning the Premier League for Man United: 14.5985401459854
Ratio of winning the Premier League for Arsenal: 9.48905109489051
Ratio of winning the Premier League for Bradford: 0.0
Ratio of winning the Premier League for Ipswich: 0.0
Ratio of winning the Premier League for Middlesbrough: 0.0
Ratio of winning the Premier League for Everton: 6.569343065693431
Ratio of winning the Premier League for Man City: 7.2992700729927
Ratio of winning the Premier League for New

We will need these calculations later because yes I know that soccer is very wild and unpredictable but for my models sake I want this to be as accurate as possible. For example yes Man United has won the Prem 20 times (unforuntately); however Man City has won 4 of the last 6 (and we are definitely winning next year) 

Next up I want to get a compiled dataframe together of all the stats/metrics that I think we be useful for predicting what type of team can win the greatest title in English football.

Some of the more advanced metrics I want to capture across all seasons include (and to me, and partly Chat GPT, are important factors for winning):
 - Shot Quality (Shots on Target)
 - Set Piece Goals (Corners, free kicks, penalties)
 - Opponent Shot Quality
 - Number of Clean Sheets
 -  Goal Differential
 - Saves Differential
 - Goal Conversion Rate
 - Number of Bookings (Red and Yellow Cards)

In [163]:
Season1 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2000-01.csv") 
Season2 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2001-02.csv")
Season3 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2002-03.csv")
Season4 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2003-04.csv")
Season5 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2004-05.csv")
Season6 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2005-06.csv")
Season7 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2006-07.csv")
Season8 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2007-08.csv")
Season9 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2008-09.csv")
Season10 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2009-10.csv")
Season11 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2011-12.csv")
Season12 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2012-13.csv")
Season13 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2013-14.csv")
Season14 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2014-15.csv")
Season15 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2015-16.csv")
Season16 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2016-17.csv")
Season17 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2017-18.csv")
Season18 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2018-19.csv")
Season19 = pd.read_csv(r"C:\Users\JosephMoffatt\OneDrive - Thought Logic\Desktop\GitHubRepos\RefurbishedCode\PremLeaguePredictor\raw_datasets\2019-20.csv")

In [166]:
ArrayOfDataFrames = []
ArrayOfDataFrames.append(Season1)
ArrayOfDataFrames.append(Season2)
ArrayOfDataFrames.append(Season3)
ArrayOfDataFrames.append(Season4)
ArrayOfDataFrames.append(Season5)
ArrayOfDataFrames.append(Season6)
ArrayOfDataFrames.append(Season7)
ArrayOfDataFrames.append(Season8)
ArrayOfDataFrames.append(Season9)
ArrayOfDataFrames.append(Season10)
ArrayOfDataFrames.append(Season11)
ArrayOfDataFrames.append(Season12)
ArrayOfDataFrames.append(Season13)
ArrayOfDataFrames.append(Season14)
ArrayOfDataFrames.append(Season15)
ArrayOfDataFrames.append(Season16)
ArrayOfDataFrames.append(Season17)
ArrayOfDataFrames.append(Season18)
ArrayOfDataFrames.append(Season19)

In [167]:
ArrayOfDataFrames

[    Div      Date       HomeTeam       AwayTeam  FTHG  FTAG FTR  HTHG  HTAG  \
 0    E0  19/08/00       Charlton       Man City     4     0   H     2     0   
 1    E0  19/08/00        Chelsea       West Ham     4     2   H     1     0   
 2    E0  19/08/00       Coventry  Middlesbrough     1     3   A     1     1   
 3    E0  19/08/00          Derby    Southampton     2     2   D     1     2   
 4    E0  19/08/00          Leeds        Everton     2     0   H     2     0   
 ..   ..       ...            ...            ...   ...   ...  ..   ...   ...   
 375  E0  19/05/01       Man City        Chelsea     1     2   A     1     1   
 376  E0  19/05/01  Middlesbrough       West Ham     2     1   H     2     1   
 377  E0  19/05/01      Newcastle    Aston Villa     3     0   H     2     0   
 378  E0  19/05/01    Southampton        Arsenal     3     2   H     0     1   
 379  E0  19/05/01      Tottenham     Man United     3     1   H     1     1   
 
     HTR  ...  HC AC  HF  AF  HO  AO  

In [169]:
for i in ArrayOfDataFrames:
    print(i.columns)

Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Attendance', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HHW',
       'AHW', 'HC', 'AC', 'HF', 'AF', 'HO', 'AO', 'HY', 'AY', 'HR', 'AR'],
      dtype='object')
Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Attendance', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HHW',
       'AHW', 'HC', 'AC', 'HF', 'AF', 'HO', 'AO', 'HY', 'AY', 'HR', 'AR'],
      dtype='object')
Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR'],
      dtype='object')
Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG',
       'HTAG', 'HTR', 'Referee', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC',
       'AC', 'HY', 'AY', 'HR', 'AR'],
      dtype='object')
Index(['Div', 'Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'F

So I figure the best way to accomplish this is to create functions to get each of these advanced metrics. 

#### Shot Quality (Shots on Target as Compared to Total Shots)